In [1]:
#usefull links
#https://www.valuepenguin.com/travel/delays-cancellations-bags-study


#data link
#https://www.kaggle.com/datasets/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018

#similar work
#https://medium.com/analytics-vidhya/modeling-flight-delays-through-u-s-flight-data-2f0b3d7e2c89

#SIMILAR WORK IMPORTANT
#https://www.tensorflow.org/tutorials/structured_data/time_series?hl=fr

#signification columns
#https://www.stat.purdue.edu/~lfindsen/stat350/airline2008_dataset_definition.pdf

#explanation delays
#https://aspm.faa.gov/aspmhelp/index/Types_of_Delay.html


#signification columns
#https://www.stat.purdue.edu/~lfindsen/stat350/airline2008_dataset_definition.pdf

#https://catsr.vse.gmu.edu/SYST660/BigDataAnalysisInAviation_AOTP.pdf

#columns explanation for the considered data
#https://www.kaggle.com/datasets/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018
#formulas
#https://www.kaggle.com/datasets/usdot/flight-delays?select=flights.csv

#signification, valid, missing mismatched in data
#https://www.kaggle.com/questions-and-answers/254937

#batch, epoch
#https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/


#batch size, sequence length
#https://www.analyticsvidhya.com/blog/2021/07/understanding-rnn-step-by-step-with-pytorch/

In [2]:
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import category_encoders as ce
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import os


#for substacting series without copies
pd.options.mode.chained_assignment = None 

import category_encoders as ce
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow 
from tensorflow import keras
from tensorflow.keras import layers

2023-08-03 00:52:52.973521: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#column  explanation
#https://www.kaggle.com/datasets/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018

#formulas
#https://www.kaggle.com/datasets/usdot/flight-delays?select=flights.csv


#FL_DATE
#Date of the flight, yy/mm/dd

#OP_CARRIER
#Airline Identifier

#OP_CARRIER_FL_NUM
#Flight Number


#ORIGIN
#Starting Airport Code

#DEST
#Destination Airport Code

#CRS_DEP_TIME
#Planned Departure Time

#DEP_TIME
#Actual Departure Time
#WHEELS_OFF - TAXI_OUT

#DEP_DELAY
#Total Delay on Departure in minutes

#TAXI_OUT
#The time duration elapsed between departure from the origin airport gate 
#and wheels off

#WHEELS_OFF
#The time point that the aircraft's wheels leave the ground

#WHEELS_ON
#The time point that the aircraft's wheels touch on the ground

#TAXI_IN
#The time duration elapsed between wheels-on and gate arrival at the destination airport

#CRS_ARR_TIME
#Planned arrival time

#ARR_TIME
#Actual Arrival Time
#WHEELS_ON+TAXI_IN

#ARR_DELAY
#Total Delay on Arrival in minutes
#ARRIVAL_TIME-SCHEDULED_ARRIVAL


#CANCELLED
#Flight Cancelled (1 = cancelled)


#CANCELLATION_CODE
#Reason for Cancellation of flight: 
#A - Airline/Carrier; B - Weather; C - National Air System; D - Security


#DIVERTED
#Aircraft landed on airport that out of schedule

#CRS_ELAPSED_TIME
#Planned time amount needed for the flight trip

#ACTUAL_ELAPSED_TIME
#AIR_TIME+TAXI_IN+TAXI_OUT

#AIR_TIME
#The time duration between wheels_off and wheels_on time

#DISTANCE
#Distance between two airports

#CARRIER_DELAY
#Delay caused by the airline in minutes

#WEATHER_DELAY
#Delay caused by weather

#NAS_DELAY
#Delay caused by air system

#SECURITY_DELAY

#LATE_AIRCRAFT_DELAY
#Delay caused by security

#Unnamed: 27
#Useless column

In [4]:
#MODEL 1 - DATAFRAMES PER ORIGIN
#WE STUDY EACH ORIGIN SEPARATLY WITHIN A DIFFERENT DATAFRAME

In [5]:
#WE AIM TO EXAMINE DELAYS REGARDING EACH ORIGIN SEPARETLY 
#AND COMPARE WITH THE RESULTS  OF THE MODEL CONSIDERING ALL THE
#ORIGINS



In [6]:
#read DataFrame from pickle file
df_2009_1= pd.read_pickle("my_df_2009_1.pkl")

In [7]:
#we  examine the first  lines
df_2009_1.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,DEP_TIME,DEP_DELAY,TAXI_OUT,TAXI_IN,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
3392,2009-01-01,UA,363,IAD,ORD,3.0,128.0,14.0,4.0,110.0,124.0,106.0,88.0,589.0
3689,2009-01-01,UA,754,SEA,ORD,5.0,28.0,23.0,6.0,16.0,225.0,213.0,184.0,1721.0
3205,2009-01-01,UA,126,LAX,ORD,12.0,47.0,13.0,5.0,39.0,235.0,227.0,209.0,1745.0
3696,2009-01-01,UA,766,ORD,BWI,45.0,201.0,12.0,3.0,174.0,112.0,85.0,70.0,622.0
3648,2009-01-01,UA,696,SEA,DEN,535.0,-5.0,17.0,10.0,-20.0,158.0,143.0,116.0,1024.0


In [8]:
# Baseline Model


#we consider as predictions the average delay per  Origin et 
#par Flight Data on  the train set. 
#we examine the MAE for the validation and test set.

In [9]:
#variable indicating the dataframe to consider
val_name_df=df_2009_1

#the data proportions for the training, validation and test set
proportion_train_set=0.5
proportion_val_set=0.25
proportion_test_set=0.25

#the number of entries 
rows_df=val_name_df.shape[0]

#the number of train data
nb_train_data=round(rows_df*proportion_train_set)

#the number of validation or test data (validation and test sets will have the same number of elements)
nb_val_or_test_data=round(rows_df*proportion_val_set)

print("number all entries df_2009_1: ",rows_df)
print("number of train data: ",nb_train_data)
print("number of validation on test data: ",nb_val_or_test_data)
print()

#the train dataframe
train_df=val_name_df[:nb_train_data]
#print("train_df.head()",train_df.head()),

#the end index for the validation set
end_ind_val_set=nb_train_data+nb_val_or_test_data

#the validation dataframe
val_df=val_name_df[nb_train_data:end_ind_val_set]

#the test dataframe
test_df=val_name_df[end_ind_val_set:]

#we verify the entries of the train, validation and test sets
#we also examine if the sum of entries of the train, validation and test set 
#are equal to the number of
#entries of the initial dataframe
train_df.shape,val_df.shape, test_df.shape, train_df.shape[0]+val_df.shape[0]+test_df.shape[0]==rows_df

number all entries df_2009_1:  369288
number of train data:  184644
number of validation on test data:  92322



((184644, 14), (92322, 14), (92322, 14), True)

In [10]:
#we recall that a negative arrival delay signifies an earlier arrival
#where a negative arrival delay signifies a later arrival

#REMARK: THIS BASELIE ALGORITHM IS REALY ELEMENTARY 
#ANOTHER MORE  SOPHISTICATED SHOULD BE DEVELOPED
#HOWEVER WE WIL USE THIS HERE TO SEE THE IMPROVEMENTS
#RETURNED BY  OTHER MORE ADVANCED MODELS THAN THE MEAN AVERAGE DELAY PER ORIGIN

#function that predicts the arival delays based on the
#mean value of the arrival delay  per Origin, computed in the train set


#var_train_df=the train dataframe which we wil use to comute the mean arrival delay per ORIGIN
#var_df_for_predictions=the dataframe of which  the Arrival Delays will be predicted
#var_name_group_feature= the mean arrival delay ont he training set wil be computed for each  value of this feature
#var_name_feature_to_predict= the feature we will predict

#it returns the MAE for the predictions

def fct_predict_ar_delay_baseline_model(
    var_train_df,
    var_df_for_predictions,
    var_name_group_feature="ORIGIN",
    var_name_feature_to_predict="ARR_DELAY"):
    
    #the mean arrival delay per origin For the training set
    var_train_df["mean"]=var_train_df.groupby([var_name_group_feature])[var_name_feature_to_predict].transform('mean')
    print(var_train_df[[var_name_group_feature,"mean"]])
    print()
    
    
    #we create a dataframe df2 with column "ORIGIN" taking values 
    #the unique values of "ORIGIN" of dataframe  var_train_df
    #and column "mean" the mean arrival  delay for this origin
    var_train_df_group=var_train_df.groupby(var_name_group_feature)
    df2 = var_train_df_group.apply(lambda x: x['mean'].unique()[0]).to_frame().reset_index()
    df2 = df2.rename(columns= {0: 'mean'})
    print("df2:",df2)
    print()
    
    #verification
    #df2.loc[df2["ORIGIN"]=="HNL"]["mean"].tolist()[0]

    #the list of unque  values for variable "ORIGIN"
    li_unique_origins=df2[var_name_group_feature].tolist()
    
    print("li_unique_origins",li_unique_origins)
    print()

    print("var_name_group_feature",var_name_group_feature)
    print()
    
    #the list of values of variable "ORIGIN" for the var_df_for_predictions set
    li_origins_var_df_for_predictions=var_df_for_predictions[var_name_group_feature].tolist()
    print("len(li_origins_var_df_for_predictions)",len(li_origins_var_df_for_predictions))
    print()
    
    #we create a list with the predictions  for the arrival delay for the validtion set
    #the prediction for the arrival delay for a given ORIGIN will be the mean value of the arrival
    #delay for that ORIGIN as computed in the training set
    mean=[df2.loc[df2[var_name_group_feature]==i]["mean"].tolist()[0] for i in li_origins_var_df_for_predictions]
    len(mean)
    var_df_for_predictions["Predictions"]=mean
    print("var_df_for_predictions[[ORIGIN,Predictions]]",var_df_for_predictions[[var_name_group_feature,"Predictions"]])
    print()
    
    
    #substract the prediction from real value
    var_df_for_predictions["diff"]=\
    abs(var_df_for_predictions["Predictions"]-var_df_for_predictions[var_name_feature_to_predict])
    print("var_df_for_predictions.head()",var_df_for_predictions.head())
    print()
    
    mae_val=var_df_for_predictions["diff"].mean()
    
    return mae_val

    
    

In [11]:
#we compute the mean absolute error for the validation set 

mae_val_set=fct_predict_ar_delay_baseline_model(
    var_train_df=train_df,
    var_df_for_predictions=val_df,
    var_name_group_feature="ORIGIN",
    var_name_feature_to_predict="ARR_DELAY")

print("MAE validation set: ",mae_val_set)
mae_val_set

        ORIGIN      mean
3392       IAD  0.186332
3689       SEA  5.996539
3205       LAX  1.591040
3696       ORD  7.595339
3648       SEA  5.996539
...        ...       ...
3152905    SFO  3.225311
3152903    ORD  7.595339
3153064    ORD  7.595339
3152811    OGG  1.635110
3152828    KOA  0.978754

[184644 rows x 2 columns]

df2:    ORIGIN      mean
0     ABQ -2.626984
1     ALB  7.220290
2     ATL  0.570642
3     AUS -1.847953
4     BDL -2.973684
..    ...       ...
79    STL -0.442529
80    STT -5.975806
81    TPA -1.500302
82    TUL  2.308468
83    TUS -7.032468

[84 rows x 2 columns]

li_unique_origins ['ABQ', 'ALB', 'ATL', 'AUS', 'BDL', 'BIL', 'BOI', 'BOS', 'BTV', 'BUF', 'BUR', 'BWI', 'BZN', 'CLE', 'CLT', 'CMH', 'CVG', 'DAY', 'DCA', 'DEN', 'DFW', 'DSM', 'DTW', 'EGE', 'EWR', 'FSD', 'GEG', 'GRR', 'HDN', 'HNL', 'IAD', 'IAH', 'ICT', 'IDA', 'IND', 'JAC', 'JAX', 'JFK', 'KOA', 'LAS', 'LAX', 'LGA', 'LIH', 'MCI', 'MCO', 'MDT', 'MHT', 'MIA', 'MSP', 'MSY', 'MTJ', 'OAK', 'OGG', 'OKC', 'OMA',

22.292157046106166

In [12]:
len(test_df["ORIGIN"].unique().tolist()),len(val_df["ORIGIN"].unique().tolist()),len(train_df["ORIGIN"].unique().tolist())

(77, 72, 84)

In [13]:
#ORIGIN "GUC" is included in the test set but not in the train set
#so there is no mean Arrival Delay for this destination
#consequntly we have to remove it from the test set in order to be able to use
#the baseline algorithm we defined

lst1=train_df["ORIGIN"].unique().tolist()
lst2=test_df["ORIGIN"].unique().tolist()

len(list(set(lst1) & set(lst2))), list(set(lst2) - set(lst1)),'GUC'in lst2


(76, ['GUC'], True)

In [14]:
#as  Origin GUC is not in the train df we remove it
test_df.drop(test_df[test_df.ORIGIN =='GUC'].index, inplace=True)

#we calcule the mean absolute error for the test set
mae_test_set=fct_predict_ar_delay_baseline_model(
    var_train_df=train_df,
    var_df_for_predictions=test_df,
    var_name_group_feature="ORIGIN",
    var_name_feature_to_predict="ARR_DELAY")

print("MAE test set: ",mae_test_set)
mae_test_set


        ORIGIN      mean
3392       IAD  0.186332
3689       SEA  5.996539
3205       LAX  1.591040
3696       ORD  7.595339
3648       SEA  5.996539
...        ...       ...
3152905    SFO  3.225311
3152903    ORD  7.595339
3153064    ORD  7.595339
3152811    OGG  1.635110
3152828    KOA  0.978754

[184644 rows x 2 columns]

df2:    ORIGIN      mean
0     ABQ -2.626984
1     ALB  7.220290
2     ATL  0.570642
3     AUS -1.847953
4     BDL -2.973684
..    ...       ...
79    STL -0.442529
80    STT -5.975806
81    TPA -1.500302
82    TUL  2.308468
83    TUS -7.032468

[84 rows x 2 columns]

li_unique_origins ['ABQ', 'ALB', 'ATL', 'AUS', 'BDL', 'BIL', 'BOI', 'BOS', 'BTV', 'BUF', 'BUR', 'BWI', 'BZN', 'CLE', 'CLT', 'CMH', 'CVG', 'DAY', 'DCA', 'DEN', 'DFW', 'DSM', 'DTW', 'EGE', 'EWR', 'FSD', 'GEG', 'GRR', 'HDN', 'HNL', 'IAD', 'IAH', 'ICT', 'IDA', 'IND', 'JAC', 'JAX', 'JFK', 'KOA', 'LAS', 'LAX', 'LGA', 'LIH', 'MCI', 'MCO', 'MDT', 'MHT', 'MIA', 'MSP', 'MSY', 'MTJ', 'OAK', 'OGG', 'OKC', 'OMA',

23.019473784171762

In [15]:
#WE NEED TO REMOVE DATFRAME 40, 81 AND ANOTHER ONE AS THEY WILL NTO BE CONSIDERED IN THE NN TO DO TI AFTERWARDS